In [4]:
using CSV, DataFrames, Statistics, Dates, Gadfly, GLM, Distributions, LinearAlgebra, DecisionTree
include("Utils.jl")

partitionTrainTest (generic function with 2 methods)

In [5]:
data_surverse = CSV.read("data/surverses.csv",missingstring="-99999")
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [6]:
data_surverse = filter(row -> month(row.DATE) > 4, data_surverse) 
data_surverse = filter(row -> month(row.DATE) < 11, data_surverse) 
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [7]:
raison = coalesce.(data_surverse[:,:RAISON],"Inconnue")
data_surverse[!,:RAISON] = raison
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


In [8]:
data_surverse = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data_surverse) 
select!(data_surverse, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [9]:
surverse_df = dropmissing(data_surverse, disallowmissing=true)
first(surverse_df,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [10]:
data_precipitations = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data_precipitations, Symbol("St-Hubert")=>:StHubert)
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


In [11]:
data_precipitations = filter(row -> month(row.date) > 4, data_precipitations) 
data_precipitations = filter(row -> month(row.date) < 11, data_precipitations) 
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing


In [47]:
data_ouvrage = CSV.read("data/ouvrages-surverses.csv",missingstring="")

# Extract speficic ouvrage
df_ouvrage = filter(row ->
    row.ID_ouvrage == "3260-01D" ||
    row.ID_ouvrage == "3350-07D" ||
    row.ID_ouvrage == "4240-01D" ||
    row.ID_ouvrage == "4350-01D" ||
    row.ID_ouvrage == "4380-01D", data_ouvrage)

df_ouvrage = df_ouvrage[:,[:ID_ouvrage, Symbol("Trop-Plein X"), Symbol("Trop-Plein Y"), Symbol("Trop-Plein Z")]]
rename!(df_ouvrage, Symbol("ID_ouvrage")=>:NO_OUVRAGE)

meanHeight = mean(skipmissing(df_ouvrage[Symbol("Trop-Plein Z")]))

# Replace missing height with it's mean
df_ouvrage[ismissing.(df_ouvrage[Symbol("Trop-Plein Z")]), Symbol("Trop-Plein Z")] .= meanHeight

df_ouvrage

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[47]:13
└ @ Core In[47]:13
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[47]:14
└ @ Core In[47]:14


,NO_OUVRAGE,Trop-Plein X,Trop-Plein Y,Trop-Plein Z
,String,Float64,Float64,Float64⍰
1,3260-01D,2.98543e5,5.05676e6,20.17
2,3350-07D,2.89801e5,5.04516e6,20.75
3,4240-01D,3.05755e5,5.05665e6,11.91
4,4350-01D,3.00503e5,5.03991e6,17.61
5,4380-01D,2.9982e5,5.03643e6,17.61


In [55]:
# Analyse exploratoire

# Faire la somme des precipitations par rapport a la date
pcp_sum = by(data_precipitations, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
           Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)

pcp_max = by(data_precipitations, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)

df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", surverse_df)

rename!(df_surve, Symbol("DATE")=>:date)
rename!(pcp_max,
    Symbol("McTavish")=>:McTavishMax,
    Symbol("Bellevue")=>:BellevueMax,
    Symbol("Assomption")=>:AssomptionMax,
    Symbol("Trudeau")=>:TrudeauMax,
    Symbol("StHubert")=>:StHubertMax)

enhanced_surve = join(df_surve, pcp_sum, on=:date, kind=:inner)
enhanced_surve = join(enhanced_surve, pcp_max, on=:date, kind=:inner)
enhanced_surve = join(enhanced_surve, df_ouvrage, on=:NO_OUVRAGE, kind=:left)

McTavish_Sum_Avg = mean(skipmissing(enhanced_surve[:,:McTavish]))
Bellevue_Sum_Avg = mean(skipmissing(enhanced_surve[:,:Bellevue]))
Assomption_Sum_Avg = mean(skipmissing(enhanced_surve[:,:Assomption]))
Trudeau_Sum_Avg = mean(skipmissing(enhanced_surve[:,:Trudeau]))
StHubert_Sum_Avg = mean(skipmissing(enhanced_surve[:,:StHubert]))

McTavish_Max_Avg = mean(skipmissing(enhanced_surve[:,:McTavishMax]))
Bellevue_Max_Avg = mean(skipmissing(enhanced_surve[:,:BellevueMax]))
Assomption_Max_Avg = mean(skipmissing(enhanced_surve[:,:AssomptionMax]))
Trudeau_Max_Avg = mean(skipmissing(enhanced_surve[:,:TrudeauMax]))
StHubert_Max_Avg = mean(skipmissing(enhanced_surve[:,:StHubertMax]))

replacement = [McTavish_Sum_Avg, Bellevue_Sum_Avg, Assomption_Sum_Avg, Trudeau_Sum_Avg, StHubert_Sum_Avg,
                McTavish_Max_Avg, Bellevue_Max_Avg, Assomption_Max_Avg, Trudeau_Max_Avg, StHubert_Max_Avg]

# Commence a 4 pour les column des precipitations
for i = 4:size(enhanced_surve,2)
    for j = 1:size(enhanced_surve, 1)
        if enhanced_surve[j,i] === missing
            enhanced_surve[j,i] = floor(replacement[i - 3])
        end
    end
end

enhanced_train_data, enhanced_test_data = partitionTrainTest(enhanced_surve, 0.7)

train_data = convert(Matrix, enhanced_train_data[:,
           [:McTavish,
            :Bellevue,
            :Assomption,
            :Trudeau,
            :StHubert,
            Symbol("Trop-Plein X"),
            Symbol("Trop-Plein Y"),
            Symbol("Trop-Plein Z")]])

train_result = convert(Array{Float64}, enhanced_train_data[:,:SURVERSE])

model = build_forest(train_result, train_data, 2, 200, 0.5, 32)


Ensemble of Decision Trees
Trees:      200
Avg Leaves: 62.57
Avg Depth:  11.495

In [56]:
# Test accuracy (Il faut trouver une meilleur maniere pour faire le cross validation)
# Cette maniere n'a pas l'air legit
test_data = convert(Matrix, enhanced_test_data[:,[:McTavish, :Bellevue, :Assomption, :Trudeau, :StHubert]])
test_result = convert(Array{Float64}, enhanced_test_data[:,:SURVERSE])
accuracy = nfoldCV_forest(test_result, test_data, 10, 2)


Fold 1
Mean Squared Error:     0.03149482737716736
Correlation Coeff:      0.8630379255114888
Coeff of Determination: 0.6811148728061803

Fold 2
Mean Squared Error:     0.0454161872685362
Correlation Coeff:      0.6405826890136616
Coeff of Determination: 0.37166221763051777

Fold 3
Mean Squared Error:     0.044824489519579944
Correlation Coeff:      0.7222775607038527
Coeff of Determination: 0.5213063525712377

Fold 4
Mean Squared Error:     0.02668395237212928
Correlation Coeff:      0.8158741632373764
Coeff of Determination: 0.6567886587477064

Fold 5
Mean Squared Error:     0.03449380395777744
Correlation Coeff:      0.8188653793046639
Coeff of Determination: 0.6507502349275033

Fold 6
Mean Squared Error:     0.0389088434197983
Correlation Coeff:      0.7536670489353879
Coeff of Determination: 0.5599917315874112

Fold 7
Mean Squared Error:     0.03185282540423211
Correlation Coeff:      0.6621432909143358
Coeff of Determination: 0.42465834113605794

Fold 8
Mean Squared Error:     0

10-element Array{Float64,1}:
 0.6811148728061803 
 0.37166221763051777
 0.5213063525712377 
 0.6567886587477064 
 0.6507502349275033 
 0.5599917315874112 
 0.42465834113605794
 0.5330403540896568 
 0.5768032940497485 
 0.5748725562434895 

In [62]:
### Retrieve test data
test = CSV.read("data/test.csv");

test_df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", test)

rename!(test_df_surve, Symbol("DATE")=>:date) 

test_df_serve = join(test_df_surve, pcp_sum, on=:date, kind=:inner)
test_df_serve = join(test_df_serve, pcp_max, on=:date, kind=:inner)
test_df_serve = join(test_df_serve, df_ouvrage, on=:NO_OUVRAGE, kind=:left)

McTavish_Sum_Avg = mean(skipmissing(test_df_serve[:,:McTavish]))
Bellevue_Sum_Avg = mean(skipmissing(test_df_serve[:,:Bellevue]))
Assomption_Sum_Avg = mean(skipmissing(test_df_serve[:,:Assomption]))
Trudeau_Sum_Avg = mean(skipmissing(test_df_serve[:,:Trudeau]))
StHubert_Sum_Avg = mean(skipmissing(test_df_serve[:,:StHubert]))

McTavish_Max_Avg = mean(skipmissing(test_df_serve[:,:McTavishMax]))
Bellevue_Max_Avg = mean(skipmissing(test_df_serve[:,:BellevueMax]))
Assomption_Max_Avg = mean(skipmissing(test_df_serve[:,:AssomptionMax]))
Trudeau_Max_Avg = mean(skipmissing(test_df_serve[:,:TrudeauMax]))
StHubert_Max_Avg = mean(skipmissing(test_df_serve[:,:StHubertMax]))

replacement = [McTavish_Sum_Avg, Bellevue_Sum_Avg, Assomption_Sum_Avg, Trudeau_Sum_Avg, StHubert_Sum_Avg,
                McTavish_Max_Avg, Bellevue_Max_Avg, Assomption_Max_Avg, Trudeau_Max_Avg, StHubert_Max_Avg]

# Commence a 3 pour les column des precipitations
for i = 3:size(test_df_serve,2)
    for j = 1:size(test_df_serve, 1)
        if test_df_serve[j,i] === missing
            test_df_serve[j,i] = floor(replacement[i - 2])
        end
    end
end

test_data = convert(Matrix, test_df_serve[:,
        [:McTavish,
            :Bellevue,
            :Assomption,
            :Trudeau,
            :StHubert,
            Symbol("Trop-Plein X"),
            Symbol("Trop-Plein Y"),
            Symbol("Trop-Plein Z")]])

n = size(test_df_serve, 1)
result = Array{Float64}(undef, n)

for i = 1:n
    result[i] = apply_forest(model, test_data[i, :])
end

Y = zeros(Int64, n)
Y[result.>.5] .= 1
Y

283-element Array{Int64,1}:
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 ⋮
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [64]:
ID = Array{String}(undef, n)
for i = 1:n
    ID[i] = string(test[i,:NO_OUVRAGE], "_", Dates.format(test[i,:DATE], "yyyy-mm-dd"))
end
YTotalBool = Array{String}(undef, n)

for i = 1:n
    if Y[i] == 0
        YTotalBool[i] = "FALSE"
    else
        YTotalBool[i] = "TRUE"
    end
end

prediction = DataFrame(ID = ID, Surverse = YTotalBool)
CSV.write("sampleSubmissionTest.csv",prediction)

"sampleSubmissionTest.csv"